# QuantLib EquityOption Example

In [1]:
import QuantLib as ql

Setup Dates:

In [2]:
calendar = ql.TARGET()
todaysDate = ql.Date(15, ql.May, 1998)
settlementDate = ql.Date(17, ql.May, 1998)
ql.Settings.instance().evaluationDate = todaysDate

Our Options:

In [3]:
option_type = ql.Option.Put
underlying = 36
strike = 40
dividendYield = 0.00
riskFreeRate = 0.06
volatility = 0.20
maturity = ql.Date(17, ql.May, 1999)
dayCounter = ql.Actual365Fixed()

In [4]:
print("Option type: ", option_type)
print("Maturity = ",maturity)
print("Underlying price = ", underlying)
print("Strike = ", strike)
print("Risk-free interest rate = ", riskFreeRate)
print("Dividend yield = ", dividendYield)
print("Volatility = ", volatility)

Option type:  -1
Maturity =  May 17th, 1999
Underlying price =  36
Strike =  40
Risk-free interest rate =  0.06
Dividend yield =  0.0
Volatility =  0.2


In [5]:
print ("{:<35}{:<14}{:<14}{:<14}".format("Method", "European", "Bermudan", "American"))

Method                             European      Bermudan      American      


In [6]:
exerciseDates = [(settlementDate + ql.Period(3*i,ql.Months)) for i in range(1,4)]

In [7]:
europeanExercise = ql.EuropeanExercise(maturity)
bermudanExercise = ql.BermudanExercise(exerciseDates)
americanExercise = ql.AmericanExercise(settlementDate, maturity)

In [8]:
underlyingH = ql.QuoteHandle(ql.SimpleQuote(underlying))

bootstrap the yield/dividend/vol curves

In [9]:
flatTermStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, riskFreeRate, dayCounter))
flatDividendTS = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, dividendYield, dayCounter))
flatVolTS = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(settlementDate, calendar, volatility, dayCounter))

In [10]:
payoff = ql.PlainVanillaPayoff(option_type, strike)

In [11]:
bsmProcess = ql.BlackScholesMertonProcess(underlyingH, flatDividendTS, flatTermStructure, flatVolTS)

options

In [12]:
europeanOption = ql.VanillaOption(payoff, europeanExercise)
bermudanOption = ql.VanillaOption(payoff, bermudanExercise)
americanOption = ql.VanillaOption(payoff, americanExercise)

## Analytic formulas:

Black-Scholes for European

In [13]:
method = "Black-Scholes"
europeanOption.setPricingEngine(ql.AnalyticEuropeanEngine(bsmProcess))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

Black-Scholes                      3.844308      N/A           N/A           


semi-analytic Heston for European

In [14]:
method = "Heston semi-analytic"
hestonProcess = ql.HestonProcess(flatTermStructure, flatDividendTS,
                              underlyingH, volatility*volatility,
                              1.0, volatility*volatility, 0.001, 0.0)
hestonModel = ql.HestonModel(hestonProcess)
europeanOption.setPricingEngine(ql.AnalyticHestonEngine(hestonModel))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

Heston semi-analytic               3.844306      N/A           N/A           


semi-analytic Bates for European

In [15]:
method = "Bates semi-analytic"
batesProcess = ql.BatesProcess(flatTermStructure, flatDividendTS,
                             underlyingH, volatility*volatility,
                             1.0, volatility*volatility, 0.001, 0.0,
                             1e-14, 1e-14, 1e-14)
batesModel = ql.BatesModel(batesProcess)
europeanOption.setPricingEngine(ql.BatesEngine(batesModel))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

Bates semi-analytic                3.844306      N/A           N/A           


Barone-Adesi and Whaley approximation for American

In [16]:
method = "Barone-Adesi/Whaley"
americanOption.setPricingEngine(ql.BaroneAdesiWhaleyEngine(bsmProcess))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "N/A", "N/A", "%2.6f"%americanOption.NPV()))

Barone-Adesi/Whaley                N/A           N/A           4.459628      


Bjerksund and Stensland approximation for American

In [17]:
method = "Bjerksund/Stensland"
americanOption.setPricingEngine(ql.BjerksundStenslandEngine(bsmProcess))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "N/A", "N/A", "%2.6f"%americanOption.NPV()))

Bjerksund/Stensland                N/A           N/A           4.453064      


Integral

In [18]:
method = "Integral"
europeanOption.setPricingEngine(ql.IntegralEngine(bsmProcess))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

Integral                           3.844309      N/A           N/A           


Finite differences

In [19]:
timeSteps = 801
method = "Finite differences"
europeanOption.setPricingEngine(ql.FDEuropeanEngine(bsmProcess, timeSteps,timeSteps-1))
bermudanOption.setPricingEngine(ql.FDBermudanEngine(bsmProcess, timeSteps,timeSteps-1))
americanOption.setPricingEngine(ql.FDAmericanEngine(bsmProcess, timeSteps,timeSteps-1))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Finite differences                 3.844342      4.219538      4.486118      


Binomial method: Jarrow-Rudd

In [20]:
method = "Binomial Jarrow-Rudd"
tree_type = "JarrowRudd"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Jarrow-Rudd               3.844132      4.220342      4.486552      


In [21]:
method = "Binomial Cox-Ross-Rubinstein"
tree_type = "CoxRossRubinstein"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Cox-Ross-Rubinstein       3.843504      4.219293      4.486415      


In [22]:
method = "Additive equiprobabilities"
tree_type = "AdditiveEQPBinomialTree"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Additive equiprobabilities         3.836911      4.215371      4.480097      


In [23]:
method = "Binomial Trigeorgis"
tree_type = "Trigeorgis"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Trigeorgis                3.843557      4.219322      4.486461      


In [24]:
method = "Binomial Tian"
tree_type = "Tian"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Tian                      3.844171      4.219246      4.486413      


In [25]:
method = "Binomial Leisen-Reimer"
tree_type = "LeisenReimer"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Leisen-Reimer             3.844308      4.219705      4.486076      


In [26]:
method = "Binomial Joshi"
tree_type = "Joshi4"
europeanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
bermudanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
americanOption.setPricingEngine(ql.BinomialVanillaEngine(bsmProcess, tree_type, timeSteps))
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "%2.6f"%bermudanOption.NPV(), "%2.6f"%americanOption.NPV()))

Binomial Joshi                     3.844308      4.219705      4.486076      


In [27]:
timeSteps = 1
method = "MC (crude)"
mcSeed = 42
mcengine1 = ql.MCEuropeanEngine(bsmProcess, "pseudorandom", timeSteps, None, False, False, None, 0.02, None, mcSeed)
europeanOption.setPricingEngine(mcengine1)
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

MC (crude)                         3.834522      N/A           N/A           


In [28]:
method = "QMC (Sobol)"
nSamples = 32768 # 2^15
mcengine2 = ql.MCEuropeanEngine(bsmProcess, "lowdiscrepancy", timeSteps, None, False, False, nSamples, None, None, mcSeed)
europeanOption.setPricingEngine(mcengine2)
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "%2.6f"%europeanOption.NPV(), "N/A", "N/A"))

QMC (Sobol)                        3.844613      N/A           N/A           


In [32]:
method = "MC (Longstaff Schwartz)"
mcengine3 = ql.MCAmericanEngine(bsmProcess, "lowdiscrepancy", 100, None, True, False, None, 0.02, None, mcSeed, 2, ql.LsmBasisSystem.Monomial, 4096)
americanOption.setPricingEngine(mcengine3)
print ("{:<35}{:<14}{:<14}{:<14}".format(method, "N/A", "N/A", "%2.6f"%americanOption.NPV()))

MC (Longstaff Schwartz)            N/A           N/A           4.502732      
